In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import datetime
pd.set_option('display.max_rows', None)

In [2]:
path = '/Users/USER/Dropbox/台北最愛排房表/最愛109年訂房房況表.xls'

In [3]:
xls = pd.ExcelFile(path)

In [4]:
# read in the slide of excel
df = pd.read_excel(xls,'109年7-12月')

# Data Cleaning

In [5]:
df = df.iloc[:18,1:].drop(1).T

In [6]:
column_name = ['days_in_week',201,
 301,
 501,
 601,
 202,
 302,
 502,
 602,
 203,
 303,
 503,
 603,
 205,
 305,
 505,
 605]

In [7]:
#get relevant column
df.columns = column_name

In [8]:
# get relevant rows
df = df.iloc[1:]

# Data Manipulation

In [9]:
#get the boolean of current relevant day
current_date_boolean = pd.Series(df.index) >= date.today()

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  


In [10]:
# resettig index to allow date to be accessed
df = df.reset_index()
df.rename(columns={'index':'date'}, inplace=True)

In [11]:
#filtering the date
df = df[current_date_boolean]

In [12]:
df.head()

,date,days_in_week,201,301,501,601,202,302,502,602,203,303,503,603,205,305,505,605
121,2020-10-30,五,莊玉平(($2000不含餐),HOANG越南移工($2300含餐),林仲甫($2500/含餐),張嘉德($2500/含餐),劉查呂($2200不含餐),NaN,NaN,NaN,陳雅明($2200不含餐),LE THI越外配($2500含餐),NaN,施建宏($2300/含餐),陳盈元($2500/含餐),張順傑($2600/含餐),石信勝($2800含餐),NaN
122,2020-10-31,六,莊玉平(($2000不含餐),HOANG越南移工($2300含餐),NaN,NaN,劉查呂($2200不含餐),NaN,NaN,NaN,陳雅明($2200不含餐),LE THI越外配($2500含餐),FooChuan Chien10/31~11/14(防疫/$2500/含餐),施建宏($2300/含餐),陳盈元($2500/含餐),張順傑($2600/含餐),石信勝($2800含餐),NaN
123,2020-11-01,日,莊玉平(($2000不含餐),HOANG越南移工($2300含餐),Ila Rosdiana11/1~11/15($2300含餐)10/23有授權2300,NaN,劉查呂($2200不含餐),NaN,黃偉銘11/1~11/15(防疫/$2500/含餐)10/16已付2500訂金,鮑啟文11/1~11/15(防疫/$2500/含餐)10/16已付2500訂金,陳雅明($2200不含餐),LE THI越外配($2500含餐),FooChuan Chien($2500/含餐),施建宏($2300/含餐),陳盈元($2500/含餐),張順傑($2600/含餐),石信勝($2800含餐),NaN
124,2020-11-02,一,莊玉平(($2000不含餐),HOANG越南移工($2300含餐),Ila Rosdiana($2300含餐),NaN,劉查呂($2200不含餐),NaN,黃偉銘($2500/含餐),鮑啟文($2500/含餐),陳雅明($2200不含餐),LE THI越外配($2500含餐),FooChuan Chien($2500/含餐),施建宏($2300/含餐),陳盈元($2500/含餐),張順傑($2600/含餐),石信勝($2800含餐),NaN
125,2020-11-03,二,莊玉平(($2000不含餐),HOANG越南移工($2300含餐),Ila Rosdiana($2300含餐),NaN,劉查呂($2200不含餐),NaN,黃偉銘($2500/含餐),鮑啟文($2500/含餐),陳雅明($2200不含餐),LE THI越外配($2500含餐),FooChuan Chien($2500/含餐),施建宏($2300/含餐),陳盈元($2500/含餐),張順傑($2600/含餐),石信勝($2800含餐),NaN


In [13]:
#get rooms
rooms = [201,
 301,
 501,
 601,
 202,
 302,
 502,
 602,
 203,
 303,
 503,
 603,
 205,
 305,
 505,
 605]

# Stratify the dates that allow reservation

In [14]:
final_room_ava = pd.DataFrame()

In [15]:
#pre-allocation of number of days possible
ava_series_econ = pd.Series([0]*df.shape[0])
ava_series_stand = pd.Series([0]*df.shape[0])
ava_series_deluxe = pd.Series([0]*df.shape[0])

In [16]:
#stratify each room for 15 days continuous vacancies
for j in range(len(rooms)):
    temp_name = df.iloc[:,j+2].name
    temp_room_vacancy = df[temp_name].isna()
    for i in range(df.shape[0]-14):
        if str(temp_name)[1:] =='01':
            if temp_room_vacancy.iloc[i:i+15].sum() == 15:
                ava_series_econ[i] += 1
        elif str(temp_name)[1:] =='05':
            if temp_room_vacancy.iloc[i:i+15].sum() == 15:
                ava_series_deluxe[i] += 1
        else:
            if temp_room_vacancy.iloc[i:i+15].sum() == 15:
                ava_series_stand[i] += 1

In [17]:
final_room_ava['date'] = list(df['date'])
final_room_ava['經濟'] = ava_series_econ
final_room_ava['標準'] = ava_series_stand
final_room_ava['豪華'] = ava_series_deluxe

In [18]:
export = (final_room_ava.set_index('date').T)

In [19]:
export

date,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,...,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31
經濟,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
標準,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
豪華,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#changing it to format friendly to excel
export.columns = pd.Series(export.columns).apply(lambda x: str(x.date().strftime('%m-%d')))

In [21]:
export.T

,經濟,標準,豪華
date,,,
10-30,0,0,0
10-31,0,0,0
11-01,0,0,0
11-02,0,0,0
11-03,0,0,0
11-04,0,0,0
11-05,0,0,0
11-06,0,0,0
11-07,0,0,0


In [22]:
export.T.to_excel('今天空房表.xls')

In [23]:
export.to_excel(r'/Users/USER/Dropbox/台北最愛排房表/今天空房表.xls')

In [24]:
export

date,10-30,10-31,11-01,11-02,11-03,11-04,11-05,11-06,11-07,11-08,...,12-22,12-23,12-24,12-25,12-26,12-27,12-28,12-29,12-30,12-31
經濟,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
標準,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
豪華,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
